# Manufacturing Equipment Maintenace Prediction

In [ ]:
!pip install tensorflow
# importing the necessary library and dependencies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#modelling
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU, PReLU, ELU, ReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import backend as K
from sklearn.metrics import confusion_matrix

#feature enginering
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer

### Loading the dataset

In [ ]:
df = pd.read_csv('Dataset.csv')
df

### Data cleaning

In [ ]:
df1=df.drop(columns=['Equipment_ID'], axis=1)

In [ ]:
df1.isnull().sum()

### Checking for multicollinearity 

In [ ]:
#correlation heatmap
numeric_data = df1.select_dtypes(include=['number'])
plt.figure(figsize=(12,8))
sns.heatmap(numeric_data.corr(), annot=True, cmap = 'Greens')
plt.title("Correlation heat map of manufacturing equipment")
plt.show()

### Removing outliers

In [ ]:
Q1 = df1['Power_Consumption'].quantile(0.25)
Q3 = df1['Power_Consumption'].quantile(0.75)
IQR= Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [ ]:
data_no_outliers_1 = (df1[(df1['Power_Consumption']>= lower_bound) & (df1['Power_Consumption']<= upper_bound)])
print("Original data shape ;", df1.shape)
print("New data without outliers ;", data_no_outliers_1.shape)

In [ ]:
df1=data_no_outliers_1

In [ ]:
df1.shape

In [ ]:
x = df1.drop(columns=['Failure_Cause'], axis=1)
y = df1['Failure_Cause']

In [ ]:
x.columns.tolist()

In [ ]:
y.value_counts()

### Feature engineering

In [ ]:
# creating column transformer
num_features = x.select_dtypes(exclude="object").columns
cat_features = x.select_dtypes(include="object").columns

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    [
      ("OneHotEncoder", categorical_transformer, cat_features),
      ("StandardScaler", numerical_transformer, num_features)  
    ]
)


### Splitting dataset for model training purposes

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train.shape, x_test.shape

In [ ]:
preprocessor.fit(x_train)
x_train_processed = preprocessor.transform(x_train)
x_test_processed = preprocessor.transform(x_test)
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

### Initializing the ANN model

In [ ]:
# inititalizing ANN
classifier = Sequential()

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=35,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

### Model building

In [ ]:
#model building
def focal_loss(gamma=2., alpha=.25):
    
    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.int32)
        num_classes = K.int_shape(y_pred)[-1]
        y_true_one_hot = tf.one_hot(y_true, depth=num_classes)
        y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
        cross_entropy = -y_true_one_hot * K.log(y_pred)
        loss_weight = y_true_one_hot * K.pow(1 - y_pred, gamma)
        if alpha is not None:
            alpha_factor = K.ones_like(y_true_one_hot) * alpha
            loss_weight *= alpha_factor
        loss = loss_weight * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

# --- Define the Model Architecture ---
def create_lean_model(num_features, num_classes):
    classifier = Sequential()
    classifier.add(layers.Dense(units=8, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)))
    classifier.add(layers.Dropout(0.3))
    classifier.add(layers.Dense(units=num_classes, activation='softmax'))
    classifier.compile(optimizer='adam', loss=focal_loss(gamma=2.0, alpha=0.25), metrics=['accuracy'])
    return classifier


NUM_FEATURES = 17
NUM_CLASSES = 3

classifier = create_lean_model(NUM_FEATURES, NUM_CLASSES)

model_history = classifier.fit(
    x_train_processed,
    y_train_encoded,
    validation_split=0.33,
    batch_size=32,
    epochs=1000,
    callbacks=early_stopping
)


### MOdel evaluation

In [ ]:
model_history.history.keys()

In [ ]:
#evaluating the model
def focal_loss(gamma=2., alpha=.25):

    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, tf.int32)
        num_classes = K.int_shape(y_pred)[-1]
        y_true_one_hot = tf.one_hot(y_true, depth=num_classes)
        y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
        cross_entropy = -y_true_one_hot * K.log(y_pred)
        loss_weight = y_true_one_hot * K.pow(1 - y_pred, gamma)
        if alpha is not None:
            alpha_factor = K.ones_like(y_true_one_hot) * alpha
            loss_weight *= alpha_factor
        loss = loss_weight * cross_entropy
        return K.sum(loss, axis=-1)
    return focal_loss_fixed

print("Evaluating the model on the test set...")

loss, accuracy = classifier.evaluate(x_test_processed, y_test_encoded)

print(f"\nFinal Test Loss: {loss:.4f}")
print(f"Final Test Accuracy: {accuracy:.4f}")


#### Confusion matrix

In [ ]:
#using confusion matrix to get more accurate model performance

def plot_confusion_matrix(classifier, x_test, y_test_encoded, class_names):
  
    y_pred_probs = classifier.predict(x_test_processed)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)
    
    y_true_classes = y_test_encoded
    
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm, 
        annot=True, 
        fmt="d", 
        cmap="Blues", 
        xticklabels=class_names, 
        yticklabels=class_names
    )
    plt.title("Confusion Matrix for Failure Prediction")
    plt.ylabel("Actual Label")
    plt.xlabel("Predicted Label")
    plt.show()

if __name__ == "__main__":
    
    CLASS_NAMES = ['Electrical Failure', 'Mechanical Failure', 'Sensor Malfunction'] 
    
plot_confusion_matrix(classifier, x_test_processed, y_test_encoded, CLASS_NAMES)
